In [20]:
from pyspark.sql import DataFrame, SparkSession, functions as f
import pandas as pd
import json
from urllib.error import URLError

# https://docs.google.com/spreadsheets/d/132SKHMoaJePu4nTlBnQwfaz3dhfJiKmJUujfYkzXMdI/edit?gid=179018892#gid=179018892

def get_evidence_datasets() -> pd.DataFrame:
    # Evidence datasets:
    spreadsheet_url = f'https://docs.google.com/spreadsheets/d/1G3zq-aJg3uKBz3VZUvGV8yYVrECtpQ9SJoGX6X3B-ms/export?format=csv'

    try:
        df = pd.read_csv(spreadsheet_url, sep=',')
    except URLError:
        df = pd.read_csv('dataset_name_mapping.csv')

    # Extract table from spreadsheet:
    return (
        df
        .loc[lambda df: df.output.notna()]
        [['output', 'datasourceId']]
    )

def get_full_croissant(gid:str) -> pd.DataFrame:
    """
    (
        pd.read_csv(full_croissant_description_url,  sep=',')
        .to_csv('full_croissant_description.csv', sep=',', index=False)
    )
    """
    # Evidence datasets:
    full_croissant_description_url = f'https://docs.google.com/spreadsheets/d/132SKHMoaJePu4nTlBnQwfaz3dhfJiKmJUujfYkzXMdI/export?gid={gid}&format=csv'

    try:
        df = pd.read_csv(full_croissant_description_url, sep=',')
    except URLError:
        print('reading from backup: full_croissant_description.csv')
        df = pd.read_csv('full_croissant_description.csv')

    # Extract table from spreadsheet:
    return df

def get_full_evidence(gid: str) -> pd.DataFrame:
    return (
        get_full_croissant(gid)
        .loc[lambda df: df.dataset_name == 'evidence']
        .assign(
            field_join = lambda x: x.field_id.str.extract(r'\/(.+)$')
        )
        .drop(['dataset_name', 'Example', 'column_name', 'field_id'], axis=1)
    )

# 25.09:
gid = '179018892'

# Get a list of evidence datasets:
dataset_list = get_evidence_datasets()

# Reading a piece of dataset representing the schema of the entire dataset:
spark = SparkSession.builder.getOrCreate()

data_folder = '/Users/dsuveges/project_data/25.09/output'




In [21]:
dataset_list.head() #to_csv('dataset_name_mapping.csv', sep=',', index=False)

,output,datasourceId
0,evidence_cancer_biomarkers,cancer_biomarkers
1,evidence_cancer_gene_census,cancer_gene_census
2,evidence_chembl,chembl
3,evidence_clingen,clingen
4,evidence_crispr,crispr


In [ ]:
# Processing a field in the spark schema:
# Processing a field in the spark schema:
def process_fields(
    fields: dict[str, str], 
    parent: str, 
    dataset_name: str,
    schema: list[dict[str,str]] | None = None,
) -> list[dict[str, str]]:
    if schema is None:
        schema = []

    # Iterating over all fields of the schema:
    for field in fields.get('fields'):

        name = field.get('name')
        # Recurse for nested structs:
        if isinstance(field['type'], dict):
            if field['type']['type'] == 'struct':
                # Call self:
                schema = schema + process_fields(
                    field['type'], 
                    f'{parent}/{name}',
                    dataset_name,
                    [],
                )
            elif isinstance(field['type']['elementType'], dict):
                # Call self:
                schema = schema + process_fields(
                    field["type"]['elementType'], 
                    f'{parent}/{name}',
                    dataset_name,
                    [],
                )
        # Capture atomic column types:
        schema.append(
            {
                'dataset_name': dataset_name,
                'field_id': f'{parent}/{name}',
                'column_name': name,
            }
        )
    # Return captured schema:
    return schema


def generate_schema_representation(
    output: str,
    datasourceId: str
) -> pd.DataFrame:

    # Read dataset:
    try:
        df = spark.read.parquet(f'{data_folder}/{output}')
    except:
        print(f'missing dataset: {data_folder}/{output}')
        return None

    # Get its schema:
    data = json.loads(df.schema.json())

    # We iterate over the schema and collect field names that we will be able to annotate on the spreadsheet:
    schema = process_fields(
        data, 
        output,
        output
    )

    # Process and return schema representation:
    return (
        pd.DataFrame(schema)
        .assign(
            field_join = lambda x: x.field_id.str.extract(r'\/(.+)$')
        )
    )



# generate_schema_representation(**dataset)    
new_evidence_collated = pd.concat([
    generate_schema_representation(**row)
    for _, row in 
    dataset_list.iterrows()
])

# print(len(new_evidence_collated))
# new_evidence_collated.head()


parent: evidence_cancer_biomarkers name: id
parent: evidence_cancer_biomarkers name: targetFromSourceId
parent: evidence_cancer_biomarkers name: diseaseFromSourceMappedId
parent: evidence_cancer_biomarkers name: biomarkerName
parent: evidence_cancer_biomarkers name: biomarkers
dict
struct
parent: evidence_cancer_biomarkers/biomarkers name: geneExpression
dict
array of struct
parent: evidence_cancer_biomarkers/biomarkers/geneExpression name: id
parent: evidence_cancer_biomarkers/biomarkers/geneExpression name: name
parent: evidence_cancer_biomarkers/biomarkers name: geneticVariation
dict
array of struct
parent: evidence_cancer_biomarkers/biomarkers/geneticVariation name: functionalConsequenceId
parent: evidence_cancer_biomarkers/biomarkers/geneticVariation name: id
parent: evidence_cancer_biomarkers/biomarkers/geneticVariation name: name
parent: evidence_cancer_biomarkers name: confidence
parent: evidence_cancer_biomarkers name: datasourceId
parent: evidence_cancer_biomarkers name: data

In [4]:
old_evidence = get_full_evidence(gid)
old_evidence.head() 

,column_description,foreign_key,bioregistry_prefix,field_join
202,Identifer of the evidence source,NaN,NaN,datasourceId
203,Open Targets target identifier,target/id,NaN,targetId
204,Origin of the variant allele,NaN,NaN,alleleOrigins
205,Inheritance patterns,NaN,NaN,allelicRequirements
206,Genetic origin of a population,NaN,NaN,ancestry


In [17]:
updated_evidence_table = (
    new_evidence_collated
    .merge(
        old_evidence, on='field_join', how='left'
    )
    # Updating missing descriptions:
    .assign(
        column_description = lambda df: df.apply(
            lambda row: 
              "Evidence quality flags" if row['column_name'] == 'qualityControls' else 
              "Effect of direction on target" if row['column_name'] == 'directionOnTarget' else
              row['column_description'],
            axis=1
        )
    )
)


updated_evidence_table.loc[lambda df: df.column_description.isna()]

,dataset_name,field_id,column_name,field_join,column_description,foreign_key,bioregistry_prefix


In [18]:
(
    pd.concat([
        updated_evidence_table,
        get_full_croissant(gid).loc[lambda df: df.dataset_name != 'evidence']
    ])
    .drop(['Example', 'field_join'], axis=1)
    .sort_values(['dataset_name', 'field_id'])
    .to_csv('annotated_new_evidence_columns.csv', index=False, sep=',')
)

In [19]:
len(pd.read_csv('annotated_new_evidence_columns.csv'))

1199

In [14]:
(
    updated_evidence_table
    .assign(
        column_description = lambda df: df.apply(
            lambda row: 
              "Evidence quality flags" if row['column_name'] == 'qualityControls' else 
              "Effect of direction on target" if row['column_name'] == 'directionOnTarget' else
              row['column_description'],
            axis=1
        )
    )
    .loc[updated_evidence_table.column_description.isna()]
)

,dataset_name,field_id,column_name,field_join,column_description,foreign_key,bioregistry_prefix
23,evidence_cancer_biomarkers,evidence_cancer_biomarkers/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
44,evidence_cancer_gene_census,evidence_cancer_gene_census/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
50,evidence_cancer_gene_census,evidence_cancer_gene_census/directionOnTarget,directionOnTarget,directionOnTarget,Effect of direction on target,NaN,NaN
69,evidence_chembl,evidence_chembl/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
75,evidence_chembl,evidence_chembl/directionOnTarget,directionOnTarget,directionOnTarget,Effect of direction on target,NaN,NaN
88,evidence_clingen,evidence_clingen/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
108,evidence_crispr,evidence_crispr/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
131,evidence_crispr_screen,evidence_crispr_screen/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
164,evidence_encore,evidence_encore/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
186,evidence_europepmc,evidence_europepmc/qualityControls,qualityControls,qualityControls,Evidence quality flags,NaN,NaN
